# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 11 2022 9:22AM,241499,10,0085851989,ISDIN Corporation,Released
1,Jul 11 2022 9:21AM,241497,10,0085851752,ISDIN Corporation,Released
2,Jul 11 2022 9:21AM,241497,10,0085851766,ISDIN Corporation,Released
3,Jul 11 2022 9:21AM,241497,10,0085851778,ISDIN Corporation,Released
4,Jul 11 2022 9:21AM,241496,10,0085851142,ISDIN Corporation,Released
5,Jul 11 2022 9:21AM,241496,10,0085851139,ISDIN Corporation,Released
6,Jul 11 2022 9:21AM,241496,10,0085851137,ISDIN Corporation,Released
7,Jul 11 2022 9:21AM,241496,10,0085851149,ISDIN Corporation,Released
8,Jul 11 2022 9:21AM,241496,10,0085851162,ISDIN Corporation,Released
9,Jul 11 2022 9:21AM,241496,10,0085851164,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,241495,Released,50
28,241496,Released,50
29,241497,Released,3
30,241499,Released,1
31,241501,Released,11


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
241495,NaN,50.0
241496,NaN,50.0
241497,NaN,3.0
241499,NaN,1.0
241501,NaN,11.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
241460,1.0,6.0
241463,0.0,2.0
241468,0.0,1.0
241471,17.0,11.0
241472,21.0,9.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
241460,1,6
241463,0,2
241468,0,1
241471,17,11
241472,21,9


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,241460,1,6
1,241463,0,2
2,241468,0,1
3,241471,17,11
4,241472,21,9


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,241460,1,6
1,241463,,2
2,241468,,1
3,241471,17,11
4,241472,21,9


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 11 2022 9:22AM,241499,10,ISDIN Corporation
1,Jul 11 2022 9:21AM,241497,10,ISDIN Corporation
4,Jul 11 2022 9:21AM,241496,10,ISDIN Corporation
54,Jul 11 2022 9:21AM,241495,10,ISDIN Corporation
104,Jul 11 2022 9:20AM,241492,10,ISDIN Corporation
105,Jul 11 2022 9:20AM,241490,10,ISDIN Corporation
155,Jul 11 2022 9:20AM,241489,10,ISDIN Corporation
205,Jul 11 2022 9:19AM,241486,10,ISDIN Corporation
255,Jul 11 2022 9:19AM,241487,10,ISDIN Corporation
257,Jul 11 2022 9:19AM,241485,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 11 2022 9:22AM,241499,10,ISDIN Corporation,,1
1,Jul 11 2022 9:21AM,241497,10,ISDIN Corporation,,3
2,Jul 11 2022 9:21AM,241496,10,ISDIN Corporation,,50
3,Jul 11 2022 9:21AM,241495,10,ISDIN Corporation,,50
4,Jul 11 2022 9:20AM,241492,10,ISDIN Corporation,,1
5,Jul 11 2022 9:20AM,241490,10,ISDIN Corporation,,50
6,Jul 11 2022 9:20AM,241489,10,ISDIN Corporation,,50
7,Jul 11 2022 9:19AM,241486,10,ISDIN Corporation,,50
8,Jul 11 2022 9:19AM,241487,10,ISDIN Corporation,,2
9,Jul 11 2022 9:19AM,241485,10,ISDIN Corporation,,50


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 11 2022 9:22AM,241499,10,ISDIN Corporation,1,
1,Jul 11 2022 9:21AM,241497,10,ISDIN Corporation,3,
2,Jul 11 2022 9:21AM,241496,10,ISDIN Corporation,50,
3,Jul 11 2022 9:21AM,241495,10,ISDIN Corporation,50,
4,Jul 11 2022 9:20AM,241492,10,ISDIN Corporation,1,
5,Jul 11 2022 9:20AM,241490,10,ISDIN Corporation,50,
6,Jul 11 2022 9:20AM,241489,10,ISDIN Corporation,50,
7,Jul 11 2022 9:19AM,241486,10,ISDIN Corporation,50,
8,Jul 11 2022 9:19AM,241487,10,ISDIN Corporation,2,
9,Jul 11 2022 9:19AM,241485,10,ISDIN Corporation,50,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 11 2022 9:22AM,241499,10,ISDIN Corporation,1,
1,Jul 11 2022 9:21AM,241497,10,ISDIN Corporation,3,
2,Jul 11 2022 9:21AM,241496,10,ISDIN Corporation,50,
3,Jul 11 2022 9:21AM,241495,10,ISDIN Corporation,50,
4,Jul 11 2022 9:20AM,241492,10,ISDIN Corporation,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 11 2022 9:22AM,241499,10,ISDIN Corporation,1,NaN
1,Jul 11 2022 9:21AM,241497,10,ISDIN Corporation,3,NaN
2,Jul 11 2022 9:21AM,241496,10,ISDIN Corporation,50,NaN
3,Jul 11 2022 9:21AM,241495,10,ISDIN Corporation,50,NaN
4,Jul 11 2022 9:20AM,241492,10,ISDIN Corporation,1,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 11 2022 9:22AM,241499,10,ISDIN Corporation,1,0.0
1,Jul 11 2022 9:21AM,241497,10,ISDIN Corporation,3,0.0
2,Jul 11 2022 9:21AM,241496,10,ISDIN Corporation,50,0.0
3,Jul 11 2022 9:21AM,241495,10,ISDIN Corporation,50,0.0
4,Jul 11 2022 9:20AM,241492,10,ISDIN Corporation,1,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4588205,366,1.0
16,241481,1,0.0
18,241483,1,0.0
19,1690338,52,43.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4588205,366,1.0
1,16,241481,1,0.0
2,18,241483,1,0.0
3,19,1690338,52,43.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,366,1.0
1,16,1,0.0
2,18,1,0.0
3,19,52,43.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,366.0
1,16,Released,1.0
2,18,Released,1.0
3,19,Released,52.0
4,10,Executing,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,18,19
Status,,,,
Executing,1.0,0.0,0.0,43.0
Released,366.0,1.0,1.0,52.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,18,19
0,Executing,1.0,0.0,0.0,43.0
1,Released,366.0,1.0,1.0,52.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,18,19
0,Executing,1.0,0.0,0.0,43.0
1,Released,366.0,1.0,1.0,52.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()